# Будем описывать задачу

Для начала считаем данные из log- файла  
Абсолютно тривиальный скритп.  
[log_reader]  

[log_reader]:https://github.com/Kiri28/moex_task/blob/master/moex/bin/log_reader.py

## Основная идея:
### Будем решать задачу динамически!
### Что нам надо: 
надо за каждую секунду посчитать количество сигналов каждого типа, и информацию о задержках для них, тоесть разницу между входным и выходным сигналом данного типа. На основе задержек для сигнала каждого типа строим статистические данные.

## Начнём строить нашу прогу
### Как будет выглядеть:
1) Считываем данные построчно приводя каждую строку в массив с данными  
2) Создаём словарь данных Seconds_mass{} по типу: тип сигнала:[набор из задержек] который будет обновлятья каждую секунду.   
3) Создаём счётчик времен start_time, который будет отслеживать секунды и вовремя обновлять массив. 

Создаём словарь очередей, где на вход поступает входной сигнал и ожидает своего ответа

In [2]:
quant={}

Необходимо чтобы словарь очередей дожидался выходных сигналов, даже если истекла секунда, поэтому словарь должен быть полностью закрыт, тоесть должно исполниться чётное число сигналов.

In [3]:
sch=0

Будем отслеживать поток сигналов

In [ ]:
for i in range(len(f)):
    st=re.split('\;|,', f[i])
    if ("P2_COD"not in f[i]) and ('SQLProxy' not in f[i]): #Проверяем нет ли у нас игнорируемых сигналов
        
        mee=[]
        if st[8].split()[0]=='type':
            mee=st[8].split()
        else:
            for j in range(4,len(st)-3):
                if st[j].split()[0]=='type':
                    mee=st[j].split()
            

        #Случай, поступает новая входная команда
        #Данные содержатся в формате [формат, время начала исполнения]
        if ( st[4].split()[-1] not in quant ) and ( int(mee[-1])<100 ):
            quant[st[4].split()[-1]]=[int(mee[-1]),datetime.datetime.strptime(st[0],'%H:%M:%S.%f')]
            sch+=1
    
        #Случай, когда поступает новая выходная команда
        if ( st[4].split()[-1] in quant ) and ( int(mee[-1])>100 ):
            quant[st[4].split()[-1]][1]=abs(quant[st[4].split()[-1]][1]-datetime.datetime.strptime(st[0],'%H:%M:%S.%f')).total_seconds()
            sch-=1

В случае истечения секунды необходимо занести данные в словарь Seconds_mass{}.  

In [ ]:
#Проверяем прошла ли секунда и если да, то заносим все данные в таблицу
        if (abs(current_time-datetime.datetime.strptime(st[0],'%H:%M:%S.%f')).total_seconds()>1 and sch==0)or(i==len(f)):
            #Теперь отсчёт начинаем с другого момента
            timing=current_time.strftime("%H.%M.%S")
            current_time=datetime.datetime.strptime(st[0],'%H:%M:%S.%f')
            
            
            
            #Теперь, когда истекла секунда, создаём словарь для последующего занесения данных в ксв (Хааа, звучит прям как кфс)))
            #Seconds_mass={} - так будут его звать
            Seconds_mass={}

            for j in quant:
                #Проверяем есть ли указанный элемент в словаре
                if quant[j][0] not in Seconds_mass:
                    Seconds_mass[quant[j][0]]=[quant[j][1]]
                else:
                    Seconds_mass[quant[j][0]].append(quant[j][1])

Теперь надо занести словарь в csv формат и очистить его   
Все основные мат. функции сделаны тут:
[math_foo]  

[math_foo]:https://github.com/Kiri28/moex_task/blob/master/moex/bin/math_foo.py

Функция связки с csv -ридером:  
[csv_writer]  

[csv_writer]:https://github.com/Kiri28/moex_task/blob/master/moex/bin/csv_writer.py

In [ ]:
 #Заносим результаты в csv
for j in Seconds_mass:
    Seconds_mass[j].sort()
    ins(timing,j,len(Seconds_mass[j]),sum(Seconds_mass[j])/len(Seconds_mass[j]),median(Seconds_mass[j]),pers_90(Seconds_mass[j]),pers_99(Seconds_mass[j]),max(Seconds_mass[j]))

В итоге все данные заносятся в csv файл 
[result-1.csv]  

[result-1.csv]:https://github.com/Kiri28/moex_task/blob/master/moex/result_1.csv
__________
### А вот и примерный вариант:

In [10]:
from pandas import read_csv
df1 = read_csv("result_1.csv")

In [11]:
df1[:10]

,time,type,quantity,medium,median,pers90,pers99,max
0,22.00.00,1,42,0.000247,0.000247,0.000276,0.000291,0.000291
1,22.00.00,3,39,0.000218,0.000212,0.000232,0.000332,0.000332
2,22.00.00,11,14,0.000228,0.000221,0.000257,0.000311,0.000311
3,22.00.00,9,15,0.000279,0.000272,0.000316,0.000324,0.000324
4,22.00.01,1,87,0.000249,0.000248,0.000276,0.000305,0.000315
5,22.00.01,3,92,0.000214,0.000210,0.000235,0.000328,0.000332
6,22.00.01,11,18,0.000228,0.000227,0.000248,0.000311,0.000311
7,22.00.01,9,18,0.000274,0.000271,0.000312,0.000324,0.000324
8,22.00.02,1,131,0.000251,0.000248,0.000274,0.000329,0.000538
9,22.00.02,3,135,0.000216,0.000211,0.000254,0.000331,0.000332
